# PANDAPROSUMER EXAMPLE: COMBINED HEAT AND POWER (CHP)

Example prepared by:
--------------------
    Izak Oberčkal Pluško, Marko Keber, Katja Klinar, Tine Seljak*, Andrej Kitanovski

    Faculty of Mechanical Engineering, University of Ljubljana, Slovenia
    
    *Contact: tine.seljak@fs.uni-lj.si

DESCRIPTION:
--------------------
This example describes how to create a single CHP element in the *pandaprosumer* software package and connect it to a single consumer. The user can choose the size of the CHP unit, while the demand data is read from a an Excel file and stored in a pandas dataframe. It includes information about the heat flow required by the consumer and the corresponding CHP output configuration (topping or bottoming). 

Glossary:
---------
- Network: a configuration of connected energy generators and energy consumers
- Element: a single energy generator or a single energy consumer
- Container: a pandaprosumer data structure that contains data of an individual element; each element must have its container 
- Controller: the logic of an element that defines its behaviour and its limits
- General controller: the first controller in the network that interacts with controllers of all other elements; this controller also manages external data
- Map / mapping: a connection between two elements; contains information about the what is exchanged between the two elements

Network design philosophy:
--------------------------
In pandaprosumer, a system's component is represented by a network element. Each element is assigned a *container* and its own *element controller*. A container is a structure that contains the component's configuration data (static input data), which can include information that will not change in the analysis such as size, nominal power, efficiency, etc. The behaviour of an element is governed by its *controller*. Connections between elements are defined in *maps*, which couple output parameters of one controller to the input parameter of a controller of a connected element. 
The network is managed by a *general controller* called *ConstProfileController*. This controller is connected to all element controllers and manages dynamic input data from external sources (e.g. Excel file). For each time step it distributes the dynamic input data to the relevant element controllers. 

CREATING A NETWORK:
--------------------
If we are not in pandaprosumer parent directory, we should add it to the path so that the program knows where to find the necessary functions:

In [ ]:
import sys
import os

current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
sys.path.append(parent_directory)

1 - INPUT DATA:
---------------
First let's import libraries required for data management.

In [ ]:
import pandas as pd
from pandapower.timeseries.data_sources.frame_data import DFData

Next, we need to define properties of the CHP element (size, name), which are treated as static input data, i.e. data (characteristics) that don't change during an analysis. Size is defined as the nominal maximum electrical power (unit: [kW]). A name can be added to describe the CHP element. # TODO opis 

In [ ]:
chp_name = 'example_chp'
chp_size_kw = 500
chp_fuel = 'ng'
altitude_m = 0

We define the analysis time series.

In [ ]:
start = '2020-01-01 00:00:00' 
end = '2020-01-02 00:00:00'
time_resolution = 900 # 15 min 

Now we import our demand data and transform it into an appropriate DFData object. All data of an individual element is stored in a dedicated DFData object.

In [ ]:
demand_data = pd.read_excel('example_data/input_chp_v2.xlsx') 

print(demand_data.head())

dur = pd.date_range(start, end, freq="15min", tz='utc')                              
demand_data.index = dur
demand_input = DFData(demand_data)

2 - CREATING ELEMENTS OF THE NETWORK:
--------------------------------------

In this example, the network is made up of two elements: a source and a consumer. The source is represented by a single CHP unit and the consumer is modelled by a single heat demand element. 

![network](figures/chp_cons_1.png)

2.1. ELEMENT 1 (SOURCE) ---> CHP ELEMENT:

First we define an empty prosumer container object which in practice can be anything. In pandaprosumer, each element of the network has its own container, which is later filled with data and results.

In [ ]:
from pandaprosumer.create import create_empty_prosumer_container

chp_prosumer = create_empty_prosumer_container()

Then we define the period of the analysis using input data about the analysis of time and also timezone and period name.

In [ ]:
from pandaprosumer.create import create_period

chp_period = create_period(chp_prosumer, time_resolution, start, end, 'utc', 'default')

Now we fill the container created above (chp_prosumer). We pass into the function of the empty prosumer container created above static input data (size, altitude). Finally, the last static input data (name) is passed to the function.

In [ ]:
from pandaprosumer.create import create_ice_chp

chp_index = create_ice_chp(chp_prosumer, chp_size_kw, chp_fuel, altitude_m, name=chp_name)
print(chp_prosumer.ice_chp)

2.2. ELEMENT 2 (CONSUMER) ---> DEMAND ELEMENT:

Here we create the second element in the network, an element that represents the energy demand. In this case we have dynamic heat demand that is defined in the Excel file and is managed by the *general controller*. Scaling parameter scales the demand appropriately.

In [ ]:
from pandaprosumer.create import create_heat_demand

heat_demand_index = create_heat_demand(chp_prosumer, scaling=1.0)

3 - CREATING CONTROLLERS OF THE ELEMENTS:
------------------------------------------

![controllers](figures/chp_cons_2.png)

3.1. DATA FOR CONTROLLERS:

The first step in creating controllers is to define where each element controller gets the necessary data. 

3.1.1. GENERAL CONTROLLER DATA:

The general controller manages data from external sources. The input columns for this controller are the columns of the Excel file where dynamic input data is stored. The output columns are used later in the definitions of the mappings. 

In [ ]:
from pandaprosumer.controller.data_model import ConstProfileControllerData

const_controller_data = ConstProfileControllerData(
    input_columns=['cycle', 'temperature_ice_chp_k', 'demand'],
    result_columns=["cycle_cp", "temperature_cp", "demand_cp"],
    period_index = chp_period
);

3.1.2. CHP CONTROLLER DATA:

In CHP data controller object we add element index of the CHP that was created when we defined CHP and period index that was created when we defined period.

In [ ]:
from pandaprosumer.controller.data_model.ice_chp import IceChpControllerData

ice_chp_controller_data = IceChpControllerData(
    element_name='ice_chp',                                                 # PM: copy of this here
    element_index=[chp_index],
    period_index=chp_period
);

3.1.3. HEAT DEMAND (CONTROLLER) CONTROLLER DATA:

Like in CHP we also add appropirate element index to heat demand data controller object that was created when heat demand was defined and again the period index.

In [ ]:
from pandaprosumer.controller.data_model.heat_demand import HeatDemandControllerData

heat_demand_controller_data = HeatDemandControllerData(
    element_name='heat_demand',                                             
    element_index=[heat_demand_index],
    period_index=chp_period
);

3.2. CREATING CONTROLLERS:

Now we can create the controllers and connect them to their respective containers and data. We also define the topology of the network by setting the elements' priority. In this example, the network has a linear configuration (the direction of energy flow goes in one direction), so only the order parameter is set:
- The *general controller* (ConstProfileController) is always the first element in the network and is the only one that reads data from external sources,
- As the source, the CHP controller is the second element (order 1) in the network,
- The final order is the demand. 

3.2.1. GENERAL CONTROLLER:

In [ ]:
from pandaprosumer.controller import ConstProfileController

ConstProfileController(
    chp_prosumer,                                           
    const_object=const_controller_data,                
    df_data=demand_input,                            
    order=0,                                     
    level=0
);                                    

3.2.2. CHP CONTROLLER:

In [ ]:
from pandaprosumer.controller import IceChpController

ice_chp_obj = IceChpController(
    chp_prosumer, 
    ice_chp_controller_data,
    order=1,                                                   
    level=0
); 

3.2.3. HEAT DEMAND (CONSUMER) CONTROLLER:

In [ ]:
from pandaprosumer.controller.models.heat_demand import HeatDemandController

HeatDemandController(
    chp_prosumer,
    heat_demand_controller_data,
    order=2,
    level=0
);

4 - CREATING CONNECTIONS (MAPS) BETWEEN THE CONTROLLERS:
---------------------------------------------------------

![connections](figures/chp_cons_3.png)

For each controller we define how it is connected to other controllers. The main parameter for the map in this case is energy flow: the output energy flow of one element is linked with the input energy flow of the connected element. 

In [ ]:
from pandaprosumer.mapping import GenericMapping

4.1. CONNECTION GENERAL CONTROLLER ---> CHP:

The general controller instructs the CHP controller what cycle to use and what demand is expected from the consumer. 

In [ ]:
GenericMapping(
    chp_prosumer,                                                   
    initiator_id=0,                                           
    initiator_column="cycle_cp",                              
    responder_id=1,                                                
    responder_column="cycle"
);  

In [ ]:
GenericMapping(
    chp_prosumer,                                                   
    initiator_id=0,                                           
    initiator_column="temperature_cp",                              
    responder_id=1,                                                
    responder_column="temperature_ice_chp_k"
);  

4.2. CONNECTION CHP ---> HEAT DEMAND (CONSUMER):

The CHP controller informs the demand controller of how much thermal power it can supply at each time step. 

In [ ]:
GenericMapping(
    chp_prosumer,                                              
    initiator_id=1,                                             
    initiator_column="p_th_out_kw",
    responder_id=2,                                           
    responder_column="q_received_kw"
);

4.3. CONNECTION GENERAL CONTROLLER ---> HEAT DEMAND (CONSUMER):

The general controller sends the demand controller information about the actual demand at each time step.

In [ ]:
GenericMapping(
    chp_prosumer,                                                  
    initiator_id=0,                                               
    initiator_column="demand_cp",
    responder_id=2,                                                
    responder_column="q_demand_kw"
);

5 - RUNNING THE ANALYSIS:
-----------------------

We can now run the analysis with the input data defined above. 

In [ ]:
chp_prosumer.period

In [ ]:
from pandaprosumer.run_time_series import run_timeseries

run_timeseries(chp_prosumer, chp_period, True);

6 - PRINTING AND PLOTTING RESULTS:
----------------------------------

First, we plot the evolution of the demand from the Excel file.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
demand_data.plot(y='demand');
plt.show()

In [ ]:
print(chp_prosumer.heat_demand)

Then we list the availabel results for the CHP element. Results are stored in the form of time series, which can be plotted on a graph. The *.time_series* command lists all input and output dataframes. 

In [ ]:
print(chp_prosumer.time_series)

Here *.data_source* lists all available dataframes that we then specify with index in *.loc[]*.

In [ ]:
print(chp_prosumer.time_series.data_source)

Before plotting we have to look at the resulting dataframe to see which quantity (column) do we want to plot. Index can be an integer or the defined name of specific component you want to look at.

In [ ]:
print(chp_prosumer.time_series.data_source.iloc[0].df.head())

In [ ]:
res_df = chp_prosumer.time_series
res_df.set_index('name', inplace=True)
print(res_df.loc[chp_name].data_source.df.head())

Now we can plot the evolution of the electrical power that the CHP generates based on the demand. 

In [ ]:
chp_prosumer.time_series.data_source.loc[chp_name].df.p_el_out_kw.plot()
plt.show()

We can also for example plot time evolution of mass flow.

In [ ]:
chp_prosumer.time_series.data_source.loc[chp_name].df.mdot_fuel_in_kg_per_s.plot()
plt.show()

We now need to remove the log file handler.

In [ ]:
ice_chp_obj.remove_logfile_handler()

ACKNOWLEDGEMENTS:
-----------------
We would like to thank Pratikshya Mohanty and Odile Capron for their help in preparing this tutorial. Special thanks also to Pawel Lytaev and colleagues from the University of Kassel for their code reviews and suggestions during the development of the models. Support from the SenergyNets project (XXXXX, No. XXXXXX) is gratefully acknowledged. \
#TODO: DOPOLNI ZGORNJI TEKST